In [46]:
%pwd

'c:\\ETE project\\Glass_prediction-using-Daghub-Mlflow'

In [45]:
%pwd
os.chdir("ETE project\Glass_prediction-using-Daghub-Mlflow")

In [47]:
import pandas as pd
data=pd.read_csv("artifacts/data_ingestion/glass.csv")
data.head()


,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type of glass
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0,1
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0,1


In [48]:
data.drop(columns=["Id number"],inplace=True)

KeyError: "['Id number'] not found in axis"

In [49]:
data.head()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type of glass
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0,1
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0,1


In [50]:
data.isnull().sum()

RI               0
Na               0
Mg               0
Al               0
Si               0
K                0
Ca               0
Ba               0
Fe               0
Type of glass    0
dtype: int64

In [51]:
data.shape

(214, 10)

In [52]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataValidationConfig:
    root_dir: Path
    unzip_data_dir: Path
    STATUS_FILE: str
    all_schema: dict

In [53]:
from src.FlaskMlops.constants import *
from src.FlaskMlops.utils.common import read_yaml,create_directories


In [54]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
        
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)
        
        
        create_directories([self.config.artifacts_root])
        
    def get_data_validation_config(self)-> DataValidationConfig:
        config=self.config.data_validation
        schema=self.schema.COLUMNS
        
        create_directories([config.root_dir])
        
        data_validation_config=DataValidationConfig(
            root_dir=config.root_dir,
            unzip_data_dir=config.unzip_data_dir,
            STATUS_FILE=config.STATUS_FILE,
            all_schema=schema
        )
        
        return data_validation_config
    
        

In [55]:
from src.FlaskMlops import logger

In [56]:
class DataValidation:
        def __init__(self,config: DataValidationConfig):
            self.config=config
            
        def validate_all_columns(self)-> bool:
            try:
                validation_status=None
                data=pd.read_csv(self.config.unzip_data_dir)
                all_cols=list(data.columns)
                
                all_schema=self.config.all_schema.keys()
                
                for col in all_cols:
                    if col not in all_schema:
                        validation_status=False
                        with open(self.config.STATUS_FILE,"w") as f:
                            f.write(f"Validation status: {validation_status}")
                    else:
                        validation_status=True
                        with open(self.config.STATUS_FILE,"w") as f:
                            f.write(f"Validation status: {validation_status}")
                
                return validation_status
            except Exception as e:
                raise e 
            
        def drop_unwanted_cols(self):
            try:
            
                data = pd.read_csv(self.config.unzip_data_dir)

                schema_cols = set(self.config.all_schema.keys())
                unwanted_cols = [col for col in data.columns if col not in schema_cols]

                if unwanted_cols:
                    data.drop(columns=unwanted_cols, inplace=True)
                    print(f"Dropped columns: {unwanted_cols}")

                    # Save cleaned data
                    data.to_csv(self.config.unzip_data_dir, index=False)
                else:
                    print("No unwanted columns found")

            except Exception as e:
                raise e
            
            
                        
                
                

In [57]:
try:
    config=ConfigurationManager()
    data_validation_config=config.get_data_validation_config()
    data_validation=DataValidation(config=data_validation_config)
    data_validation.drop_unwanted_cols()
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2025-12-27 13:01:04,850: INFO: common: Yaml file: config\config.yaml loaded successfully]
[2025-12-27 13:01:04,859: INFO: common: Yaml file: params.yaml loaded successfully]
[2025-12-27 13:01:04,864: INFO: common: Yaml file: schema.yaml loaded successfully]
[2025-12-27 13:01:04,871: INFO: common: Created directory at: artifacts]
[2025-12-27 13:01:04,872: INFO: common: Created directory at: artifacts/data_validation]
No unwanted columns found
